<b>
<p>
<center>
<font size="9">
Weather Image Recognition
</font>
</center>
</p>

<p>
<center>
<font size="5">
using
</font>
</center>
</p>

<p>
<center>
<font size="6">
 Convolutional Neural Networks(CNNs)
</font>
</center>
</p>
    
<p>
<center>
<font size="4">
Machine Learning I
</font>
</center>
</p>

<p>
<center>
<font size="3">
Data Science, Columbian College of Arts & Sciences, George Washington University
</font>
</center>
</p>

<p>
<center>
<font size="4">
TEAM:
</font>
<p></p>
<font size="5">
Ramana Bhaskar Kosuru(G44367009)
</font>
<p></p>
<font size="5">
Chaya Chandana Doddaiggaluru Appajigowda(G)
</font>
<p></p>
<font size="5">
Sirisha Ginnu(G)
</font>
<p></p>
</center>
</p>
</b>

# Overview

- This notebook includes code for Convolutional Neural Networks used to classify weather images.
- Here we will work on the Kaggle Dataset [weather_image_Recognition](https://www.kaggle.com/datasets/jehanbhathena/weather-dataset).
- The goal of this project:


# Notebook Configuration

## Google drive

In [ ]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/Colab Notebooks/gwu-machine_learning_I-main/'

# Get the absolute path of the deep utilities folder
abspath_util_deep = '/content/drive/My Drive/Colab Notebooks/gwu-machine_learning_I-main/code/utilities/p3_deep_learning/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Warning

In [ ]:

import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

## Matplotlib

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Set matplotlib sizes
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=20)

## TensorFlow

In [ ]:
# The magic below allows us to use tensorflow version 2.x
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

# Print TensorFlow version to verify
print(tf.__version__)


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.17.1


## Random seed

In [ ]:
# The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
import numpy as np
np.random.seed(random_seed)

# Data Preprocessing

In [ ]:
# Change working directory to the absolute path of the deep utilities folder
%cd $abspath_util_deep

# Import the deep utitilities
%run pmlm_utilities_deep.ipynb

/content/drive/My Drive/Colab Notebooks/gwu-machine_learning_I-main/code/utilities/p3_deep_learning


## Loading the data

In this project, we will work on the [weather_image_recognition](https://www.kaggle.com/datasets/jehanbhathena/weather-dataset/data) dataset.

### Creating the directory for the data

In [ ]:
import os

# Make directory
directory = os.path.dirname(abspath_curr + 'dataset/')
if not os.path.exists(directory):
    os.makedirs(directory)

### Downloading the data to the directory

In [ ]:

# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=abspath_curr + '/dataset/',
    image_size=(256, 256),  # Resize images to 224x224
    batch_size=None,
    label_mode='int',  # Labels are returned as integers
    shuffle=True  # Shuffle the dataset before splitting
)
print("\nThe length of the dataset is ", len(dataset))

Found 6862 files belonging to 11 classes.

The length of the dataset is  6862


## Getting the name of the target

In [ ]:
target = 'label'

## Getting the info

In [ ]:
# Display class names

classes = dataset.class_names

print(classes)


['dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'sandstorm', 'snow']


In [ ]:
# Get the number of classes
n_classes = len(classes)

# Print the number of classes
len(classes)


11

## Getting the training, validation and test data

In [ ]:

# Get the total number of images
total_size = len(dataset)

print(total_size)

6862


In [ ]:
# Calculate the sizes for Train (70%), Validation (15%), and Test (15%)
train_size = int(0.70 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size  # The remaining 15%

# Split the dataset using random_split
data_train = dataset.take(train_size)
data_rest = dataset.skip(train_size)
data_valid = data_rest.take(val_size)
data_test = data_rest.skip(val_size)



In [ ]:
# Print the sizes of each split
print(f"Total images: {total_size}")
print(f"Train size: {len(data_train)}")
print(f"Validation size: {len(data_valid)}")
print(f"Test size: {len(data_test)}")

Total images: 6862
Train size: 4803
Validation size: 1029
Test size: 1030


## Resizing the data for pretrained model

In [ ]:
# Set the default input size for the pretrained model
global input_size
input_size = [300, 300]

In [ ]:
def resize(image, label):
    image = tf.image.resize(image, input_size)  # Resize image to input size
    return image, label

# Resize the training data
data_train = data_train.map(resize)

# Resize the validation data
data_valid = data_valid.map(resize)

# Resize the test data
data_test = data_test.map(resize)

## Preprocessing the data using pretrained model

In [ ]:
# Set the preprocess_input of the pretrained model

# Importing MobileNetV2 and preprocess_input

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input



In [ ]:
# Define the preprocessing function
def preprocess_pretrain(image, label):

    # Resize the image to 256x256
    image = tf.image.resize(image, (256, 256))

    # Normalize the image
    image = keras.applications.efficientnet.preprocess_input(image)

    return image, label


# Preprocess the training data using pretrained model

data_train = data_train.map(preprocess_pretrain)

# Preprocess the validation data using pretrained model
data_valid = data_valid.map(preprocess_pretrain)

# Preprocess the test data using pretrained model
data_test = data_test.map(preprocess_pretrain)

## Shuffling, batching and prefetching the data

In [ ]:
# Shuffling the training data
data_train = data_train.shuffle(buffer_size=1000, seed=random_seed)

# Set the batch size
batch_size = 32

# Batch and prefetch the training data
data_train = data_train.batch(batch_size).prefetch(1)

# Batch and prefetch the validation data
data_valid = data_valid.batch(batch_size).prefetch(1)

# Batch and prefetch the test data
data_test = data_test.batch(batch_size).prefetch(1)

# Training

## Creating the directory for the model

In [ ]:
# Make directory
directory = os.path.dirname(abspath_curr + '/result/model/')
if not os.path.exists(directory):
    os.makedirs(directory)

## Building the architecture of the model

In [ ]:

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model, optimizers

# Load the pretrained MobileNetV2 model layers
pretrained_model = MobileNetV2(include_top=False, weights='imagenet')

# Add a GlobalAveragePooling2D layer
average_pooling = layers.GlobalAveragePooling2D()(pretrained_model.output)

# Add Dropout layer to prevent overfitting
dropout = layers.Dropout(0.5)(average_pooling)

# Add the Dense output layer with softmax activation
output = layers.Dense(n_classes, activation='softmax')(dropout)

# Create the final model
model = Model(inputs=pretrained_model.input, outputs=output)

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
model.summary()



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, None, 3)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, None, None, 32) │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, None, None, 32) │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, None, None, 32) │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, None, None, 32) │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, None, None, 32) │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, None, None, 32) │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, None, None, 16) │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, None, None, 16) │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, None, None, 96) │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, None, None, 96) │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, None, None, 96) │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, None, None, 96) │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, None, None, 96) │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,272,075 (8.67 MB)

 Trainable params: 2,237,963 (8.54 MB)

 Non-trainable params: 34,112 (133.25 KB)

## Freezing the pretrained layers

In [ ]:
# For each layer in the pretrained model
for layer in pretrained_model.layers:
    # Freeze the layer
    layer.trainable = False

## Setting Callbacks

In [ ]:
# ModelCheckpoint callback
model_checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath=abspath_curr + '/result/model/model.weights.h5',
                                                      save_best_only=True,
                                                      save_weights_only=True)

# EarlyStopping callback
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,
                                                  restore_best_weights=True)

# ReduceLROnPlateau callback
reduce_lr_on_plateau_cb = keras.callbacks.ReduceLROnPlateau(factor=0.1,
                                                            patience=1)

## Compiling the model
I used the default learning rate of Adam optimizer but the accuracy was less. So I changed the learning rate to 0.001

In [ ]:
#Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

## Training the model

In [ ]:
# Train, evaluate and save the best model
history = model.fit(data_train,
                    epochs=20,
                    validation_data=data_valid,
                    callbacks=[model_checkpoint_cb,
                               early_stopping_cb,
                               reduce_lr_on_plateau_cb])

Epoch 1/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 1031s 6s/step - accuracy: 0.3787 - loss: 2.2975 - val_accuracy: 0.5850 - val_loss: 1.2808 - learning_rate: 0.0100
Epoch 2/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 472s 3s/step - accuracy: 0.5348 - loss: 1.6854 - val_accuracy: 0.6443 - val_loss: 1.1214 - learning_rate: 0.0100
Epoch 3/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 434s 3s/step - accuracy: 0.5721 - loss: 1.6068 - val_accuracy: 0.6035 - val_loss: 1.2294 - learning_rate: 0.0100
Epoch 4/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 487s 3s/step - accuracy: 0.5803 - loss: 1.4837 - val_accuracy: 0.7036 - val_loss: 0.9066 - learning_rate: 1.0000e-03
Epoch 5/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 497s 3s/step - accuracy: 0.6224 - loss: 1.2925 - val_accuracy: 0.6910 - val_loss: 0.9221 - learning_rate: 1.0000e-03
Epoch 6/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 506s 3s/step - accuracy: 0.6402 - loss: 1.1960 - val_accuracy: 0.7279 - val_loss: 0.8602 - learning_rate: 1.0000e-04
Epoch 7/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 471s 3s/step - accuracy: 0.64

## Plotting the learning curve

### Creating the directory for the figure

In [ ]:
# Make directory
directory = os.path.dirname(abspath_curr + '/result/figure/')
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
import pandas as pd

# Create a figure
pd.DataFrame(history.history).plot(figsize=(8, 5))

# Set grid
plt.grid(True)

# Save and show the figure
plt.tight_layout()
plt.savefig(abspath_curr + '/result/figure/learning_curve_before_unfreezing.pdf')
plt.show()

## Unfreezing the pretrained layers

In [ ]:
# For each layer in the pretrained model
for layer in pretrained_model.layers:
    # Unfreeze the layer
    layer.trainable = True

## Compiling the model
Here we use a lower learning rate (by a factor of 10) of Adam optimizer, so that it is less likely to compromise the pretrained weights.

I adjusted it from 0.001 to 0.0001 because of the low test accuracy.

In [ ]:
# Compile the model
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Training the model

In [ ]:
# Train, evaluate and save the best model
history = model.fit(data_train,
                    epochs=15,
                    validation_data=data_valid,
                    callbacks=[model_checkpoint_cb,
                               early_stopping_cb,
                               reduce_lr_on_plateau_cb])

## Plotting the learning curve

In [ ]:
# Create a figure
pd.DataFrame(history.history).plot(figsize=(8, 5))

# Set grid
plt.grid(True)

# Save and show the figure
plt.tight_layout()
plt.savefig(abspath_curr + '/result/figure/learning_curve_after_unfreezing.pdf')
plt.show()

# Testing

## Loading the saved model

In [ ]:
# Load the saved model
model.load_weights(filepath=abspath_curr + '/result/model/model.weights.h5')

## Evaluating the model

In [ ]:
loss, accuracy = model.evaluate(data_test)